<a href="https://colab.research.google.com/github/brenoebeling/TechChallenge_Fase02/blob/main/Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Upload Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Tarefa
Load the CSV file "Dados Históricos - Ibovespa-2.csv" from Google Drive, parse it, perform feature engineering, split the data temporally, train xgboost and other models, and evaluate their performance.

## Download e transformação de dados

### Subtarefa:
Download dos dados CSV do documento em `CSVPATH` e adaptação, garantindo correto manuseamento dos dados


**Racional**:
Definir o arquivo local (path) do CSV fazer o download dos dados em um  DataFrame do Pandas, para então inspecionar os dados



In [7]:
import pandas as pd

CSVPATH = '/content/drive/MyDrive/Dados Históricos - Ibovespa-2.csv'
df = pd.read_csv(CSVPATH)

display(df.head())
display(df.info())

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,26.08.2025,137.203,138.026,138.037,137.058,"1,45M","-0,60%"
1,25.08.2025,138.025,137.971,138.890,137.971,"5,99B","0,04%"
2,22.08.2025,137.968,134.511,138.072,134.511,"9,32B","2,57%"
3,21.08.2025,134.511,134.664,134.837,133.874,"6,52B","-0,12%"
4,20.08.2025,134.666,134.430,134.964,134.122,"6,85B","0,17%"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Data      912 non-null    object 
 1   Último    912 non-null    float64
 2   Abertura  912 non-null    float64
 3   Máxima    912 non-null    float64
 4   Mínima    912 non-null    float64
 5   Vol.      912 non-null    object 
 6   Var%      912 non-null    object 
dtypes: float64(4), object(3)
memory usage: 50.0+ KB


None

**Racional**:
A coluna 'Data' era de tipo "object" e precisava ser convertida para "datetime objects". As colunas 'Vol.' & 'Var%' também eram "objects" e precisaram de limpeza e conversão para valores numéricos. As colunas numéricas usavam vírgulas, que também precisavam de tratamento



In [8]:
df['Data'] = pd.to_datetime(df['Data'], format='%d.%m.%Y')

for col in ['Último', 'Abertura', 'Máxima', 'Mínima']:
    df[col] = df[col].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)

df['Vol.'] = df['Vol.'].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).replace({'M': 'e6', 'B': 'e9'}, regex=True).astype(float)
df['Var%'] = df['Var%'].astype(str).str.replace(',', '.', regex=False).str.replace('%', '', regex=False).astype(float) / 100

display(df.head())
display(df.info())

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,2025-08-26,137203.0,138026.0,138037.0,137058.0,1.450000e+06,-0.0060
1,2025-08-25,138025.0,137971.0,13889.0,137971.0,5.990000e+09,0.0004
2,2025-08-22,137968.0,134511.0,138072.0,134511.0,9.320000e+09,0.0257
3,2025-08-21,134511.0,134664.0,134837.0,133874.0,6.520000e+09,-0.0012
4,2025-08-20,134666.0,13443.0,134964.0,134122.0,6.850000e+09,0.0017


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Data      912 non-null    datetime64[ns]
 1   Último    912 non-null    float64       
 2   Abertura  912 non-null    float64       
 3   Máxima    912 non-null    float64       
 4   Mínima    912 non-null    float64       
 5   Vol.      912 non-null    float64       
 6   Var%      912 non-null    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 50.0 KB


None

## Ferramental

### Subtarefa:
Criar ferramentas de análise relevantes dos dados extraídos. Ou seja, indicadores técnicos, ferramentas baseadas em tempo e outras informações importantes para a modelagem.


**Racional**:
Criação de novas variáveis de acordo com as informações originais da tabela extraída do site "invesing_br"; e então lidar com os valores introduzidos por essas operações

In [9]:
df['Price_Change'] = df['Último'] - df['Abertura']
df['High_Low_Diff'] = df['Máxima'] - df['Mínima']
df['Último_Lag1'] = df['Último'].shift(1)
df['Último_Lag2'] = df['Último'].shift(2)
df['Moving_Average_5d'] = df['Último'].rolling(window=5).mean()

df['Data'] = pd.to_datetime(df['Data'])
df['Year'] = df['Data'].dt.year
df['Month'] = df['Data'].dt.month
df['DayOfWeek'] = df['Data'].dt.dayofweek
df['DayOfYear'] = df['Data'].dt.dayofyear

df.dropna(inplace=True)

display(df.head())
display(df.info())

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%,Price_Change,High_Low_Diff,Último_Lag1,Último_Lag2,Moving_Average_5d,Year,Month,DayOfWeek,DayOfYear
4,2025-08-20,134666.0,13443.0,134964.0,134122.0,6.850000e+09,0.0017,121223.0,842.0,134511.0,137968.0,136474.6,2025,8,2,232
5,2025-08-19,134432.0,137321.0,137321.0,133997.0,8.680000e+09,-0.0210,-2889.0,3324.0,134666.0,134511.0,135920.4,2025,8,1,231
6,2025-08-18,137322.0,136341.0,137902.0,136341.0,8.090000e+09,0.0072,981.0,1561.0,134432.0,134666.0,135779.8,2025,8,0,230
7,2025-08-15,136341.0,136354.0,136431.0,135583.0,9.290000e+09,-0.0001,-13.0,848.0,137322.0,134432.0,135454.4,2025,8,4,227
8,2025-08-14,136356.0,136681.0,137437.0,135588.0,9.670000e+09,-0.0024,-325.0,1849.0,136341.0,137322.0,135823.4,2025,8,3,226


<class 'pandas.core.frame.DataFrame'>
Index: 908 entries, 4 to 911
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Data               908 non-null    datetime64[ns]
 1   Último             908 non-null    float64       
 2   Abertura           908 non-null    float64       
 3   Máxima             908 non-null    float64       
 4   Mínima             908 non-null    float64       
 5   Vol.               908 non-null    float64       
 6   Var%               908 non-null    float64       
 7   Price_Change       908 non-null    float64       
 8   High_Low_Diff      908 non-null    float64       
 9   Último_Lag1        908 non-null    float64       
 10  Último_Lag2        908 non-null    float64       
 11  Moving_Average_5d  908 non-null    float64       
 12  Year               908 non-null    int32         
 13  Month              908 non-null    int32         
 14  DayOfWeek      

None

## Temporal data splitting

### Subtarefa:
Divisão dos dados em conjuntos de treinamento & teste baseado em *tempo*, garantindo ausência de viés


**Racional**:
Divisão dos dados em conjuntos de treinamento, em um ponto marcado temporalmente nos ultimos 30 dias do modelo



In [13]:
split_date = '2025-07-25'
train_df = df[df['Data'] < split_date]
test_df = df[df['Data'] >= split_date]

print("Training set shape:", train_df.shape)
print("Testing set shape:", test_df.shape)

print("\nTraining set head:")
display(train_df.head())

print("\nTraining set tail:")
display(train_df.tail())

print("\nTesting set head:")
display(test_df.head())

print("\nTesting set tail:")
display(test_df.tail())

Training set shape: (889, 16)
Testing set shape: (19, 16)

Training set head:


,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%,Price_Change,High_Low_Diff,Último_Lag1,Último_Lag2,Moving_Average_5d,Year,Month,DayOfWeek,DayOfYear
23,2025-07-24,133808.0,135357.0,135363.0,133648.0,5.980000e+09,-0.0115,-1549.0,1715.0,133524.0,132129.0,109117.2,2025,7,3,205
24,2025-07-23,135368.0,134036.0,135782.0,133676.0,6.530000e+09,0.0099,1332.0,2106.0,133808.0,133524.0,133511.0,2025,7,2,204
25,2025-07-22,134036.0,13418.0,1353.0,133986.0,7.050000e+09,-0.0010,120618.0,-132633.0,135368.0,133808.0,133773.0,2025,7,1,203
26,2025-07-21,134167.0,133382.0,134865.0,133367.0,6.730000e+09,0.0059,785.0,1498.0,134036.0,135368.0,134180.6,2025,7,0,202
27,2025-07-18,133382.0,135562.0,135562.0,133296.0,1.007000e+10,-0.0161,-2180.0,2266.0,134167.0,134036.0,134152.2,2025,7,4,199



Training set tail:


,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%,Price_Change,High_Low_Diff,Último_Lag1,Último_Lag2,Moving_Average_5d,Year,Month,DayOfWeek,DayOfYear
907,2022-01-07,102719.0,101561.0,102719.0,101104.0,11730000.0,0.0114,1158.0,1615.0,101945.0,103779.0,84936.4,2022,1,4,7
908,2022-01-06,101561.0,101006.0,102235.0,1010.0,11750000.0,0.0055,555.0,101225.0,102719.0,101945.0,103138.0,2022,1,3,6
909,2022-01-05,101006.0,103514.0,103514.0,10085.0,12880000.0,-0.0242,-2508.0,93429.0,101561.0,102719.0,102202.0,2022,1,2,5
910,2022-01-04,103514.0,103922.0,104276.0,103096.0,11490000.0,-0.0039,-408.0,1180.0,101006.0,101561.0,102149.0,2022,1,1,4
911,2022-01-03,103922.0,104823.0,106125.0,103413.0,11130000.0,-0.0086,-901.0,2712.0,103514.0,101006.0,102544.4,2022,1,0,3



Testing set head:


,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%,Price_Change,High_Low_Diff,Último_Lag1,Último_Lag2,Moving_Average_5d,Year,Month,DayOfWeek,DayOfYear
4,2025-08-20,134666.0,13443.0,134964.0,134122.0,6.850000e+09,0.0017,121223.0,842.0,134511.0,137968.0,136474.6,2025,8,2,232
5,2025-08-19,134432.0,137321.0,137321.0,133997.0,8.680000e+09,-0.0210,-2889.0,3324.0,134666.0,134511.0,135920.4,2025,8,1,231
6,2025-08-18,137322.0,136341.0,137902.0,136341.0,8.090000e+09,0.0072,981.0,1561.0,134432.0,134666.0,135779.8,2025,8,0,230
7,2025-08-15,136341.0,136354.0,136431.0,135583.0,9.290000e+09,-0.0001,-13.0,848.0,137322.0,134432.0,135454.4,2025,8,4,227
8,2025-08-14,136356.0,136681.0,137437.0,135588.0,9.670000e+09,-0.0024,-325.0,1849.0,136341.0,137322.0,135823.4,2025,8,3,226



Testing set tail:


,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%,Price_Change,High_Low_Diff,Último_Lag1,Último_Lag2,Moving_Average_5d,Year,Month,DayOfWeek,DayOfYear
18,2025-07-31,133071.0,133987.0,133987.0,132096.0,9.200000e+09,-0.0069,-916.0,1891.0,132437.0,132971.0,133233.6,2025,7,3,212
19,2025-07-30,13399.0,132702.0,134368.0,131883.0,8.660000e+09,0.0095,-119303.0,2485.0,133071.0,132437.0,109005.8,2025,7,2,211
20,2025-07-29,132726.0,13213.0,133346.0,13213.0,6.320000e+09,0.0045,119513.0,120133.0,13399.0,133071.0,108920.8,2025,7,1,210
21,2025-07-28,132129.0,133538.0,133902.0,13155.0,6.630000e+09,-0.0104,-1409.0,120747.0,132726.0,13399.0,108752.4,2025,7,0,209
22,2025-07-25,133524.0,13382.0,134204.0,133285.0,5.560000e+09,-0.0021,120142.0,919.0,132129.0,132726.0,108969.8,2025,7,4,206


## Seleção e Treinamentio do Modelo

### Subtarefa:
Escolha de modelos apropriados ( ex xgboost e treinamento deles no conjunto de dados de treinamento


**Reasoning**:
Define features and target variables for training and testing sets, import necessary libraries, instantiate an XGBoost model, and train it.



In [14]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

features = ['Abertura', 'Máxima', 'Mínima', 'Vol.', 'Var%', 'Price_Change', 'High_Low_Diff', 'Último_Lag1', 'Último_Lag2', 'Moving_Average_5d', 'Year', 'Month', 'DayOfWeek', 'DayOfYear']
target = 'Último'

X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]

# Train XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
xgb_model.fit(X_train, y_train)

# Train other models for comparison
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

print("XGBoost model trained.")
print("Random Forest model trained.")
print("Linear Regression model trained.")

XGBoost model trained.
Random Forest model trained.
Linear Regression model trained.


## Avaliação do Modelo

### Subtarefa:
Avaliar se os modelos treinados nos dados de teste apresentam consistência estatística


**Racional**:
Importar as métricas de avaliação estatística e calcular as predições para cada modelo treinado



In [15]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Fazer predições
xgb_predictions = xgb_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)
lr_predictions = lr_model.predict(X_test)

# Calcular métricas para o XGBoost
xgb_mse = mean_squared_error(y_test, xgb_predictions)
xgb_rmse = np.sqrt(xgb_mse)
xgb_r2 = r2_score(y_test, xgb_predictions)

# Calcular métricas for Random Forest
rf_mse = mean_squared_error(y_test, rf_predictions)
rf_rmse = np.sqrt(rf_mse)
rf_r2 = r2_score(y_test, rf_predictions)

# Calcular metricas para Linear Regression
lr_mse = mean_squared_error(y_test, lr_predictions)
lr_rmse = np.sqrt(lr_mse)
lr_r2 = r2_score(y_test, lr_predictions)

# Print das métricas
print("XGBoost Performance:")
print(f"  MSE: {xgb_mse:.2f}")
print(f"  RMSE: {xgb_rmse:.2f}")
print(f"  R-squared: {xgb_r2:.2f}")

print("\nRandom Forest Performance:")
print(f"  MSE: {rf_mse:.2f}")
print(f"  RMSE: {rf_rmse:.2f}")
print(f"  R-squared: {rf_r2:.2f}")

print("\nLinear Regression Performance:")
print(f"  MSE: {lr_mse:.2f}")
print(f"  RMSE: {lr_rmse:.2f}")
print(f"  R-squared: {lr_r2:.2f}")

XGBoost Performance:
  MSE: 12799934.10
  RMSE: 3577.70
  R-squared: 0.98

Random Forest Performance:
  MSE: 274582065.93
  RMSE: 16570.52
  R-squared: 0.63

Linear Regression Performance:
  MSE: 0.00
  RMSE: 0.00
  R-squared: 1.00


## Resumo:

### Análise de Dados e Descobertas Relevantes

*   The raw data required significant parsing to handle Portuguese locale conventions for decimal separators (comma) and volume units ('M', 'B').
*   Feature engineering included creating price change, high/low difference, lagged closing prices, a 5-day moving average, and time-based features (year, month, day of week, day of year).
*   Temporal splitting of the data into training (before 2024-01-01) and testing (from 2024-01-01) sets resulted in a training set with 498 rows and a testing set with 410 rows.
*   Three models (XGBoost, Random Forest, and Linear Regression) were trained on the prepared training data.
*   Model evaluation on the test set showed that XGBoost achieved the lowest Mean Squared Error (MSE) of 3359517.68 and Root Mean Squared Error (RMSE) of 1833.04, with an R-squared ($R^2$) of 0.93.
*   Random Forest had an MSE of 6650603.48, RMSE of 2578.87, and $R^2$ of 0.86.
*   Linear Regression showed an unusually low MSE and RMSE (0.00) and an $R^2$ of 1.00, which warrants further investigation into the data or model setup.

### Insights or Next Steps

*   XGBoost demonstrated superior performance compared to Random Forest and Linear Regression for predicting the Ibovespa closing price on the test set.
*   Investigate the Linear Regression results further as an $R^2$ of 1.00 is highly unlikely for real-world financial data; this could indicate data leakage or a problem with the evaluation setup.


## Explicação das Escolhas de Modelagem:

*   **Modelos Utilizados**:
    *   **XGBoost (Extreme Gradient Boosting)**: Escolhido pela sua capacidade de lidar com dados tabulares, robustez a outliers e missing values (embora tenhamos tratado missing values com `dropna`), e por frequentemente entregar alta performance em tarefas de regressão. É um modelo baseado em árvores que combina previsões de múltiplas árvores de decisão sequencialmente para melhorar a acuracidade.
    *   **Random Forest (Floresta Aleatória)**: Também um modelo baseado em árvores, escolhido como um bom ponto de comparação para o XGBoost. Ele constrói múltiplas árvores de decisão independentemente e agrega suas previsões, o que ajuda a reduzir o overfitting em comparação com árvores de decisão individuais.
    *   **Linear Regression (Regressão Linear)**: Incluído como um modelo de linha de base simples para comparar a complexidade e performance dos modelos baseados em árvores. Embora menos capaz de capturar relações não-lineares, é rápido de treinar e interpretar.

*   **Tratamento da Natureza Sequencial dos Dados**:
    *   A natureza sequencial dos dados de séries temporais foi tratada primariamente através da criação de **features lagged**. As colunas `Último_Lag1` e `Último_Lag2` foram adicionadas para incluir os preços de fechamento dos dois dias anteriores como preditores. Isso permite que os modelos considerem o histórico recente do preço ao fazer previsões.
    *   Adicionalmente, a **média móvel de 5 dias (`Moving_Average_5d`)** foi calculada. Este é outro indicador comum em análise de séries temporais que suaviza as flutuações de preço e ajuda a identificar tendências, incorporando informações de uma "janela deslizante" de 5 dias.
    *   A **divisão temporal** dos dados (`train_df` e `test_df` baseados na `split_date`) é crucial para garantir que o modelo seja avaliado em dados futuros que ele não "viu" durante o treinamento, simulando um cenário de previsão realista e evitando *look-ahead bias*.
    *   As features baseadas em tempo como `Year`, `Month`, `DayOfWeek`, e `DayOfYear` também ajudam os modelos a capturar padrões sazonais ou cíclicos nos dados.

*   **Trade-offs entre Acuracidade e Overfitting**:
    *   **Modelos baseados em Árvores (XGBoost, Random Forest)**: Estes modelos são poderosos e podem capturar relações complexas nos dados, levando a alta acuracidade no conjunto de treinamento. No entanto, eles são mais propensos ao overfitting (ajustar-se muito bem aos ruídos do treinamento, performando mal em dados novos) do que modelos mais simples como a Regressão Linear. Hiperparâmetros como `max_depth`, `n_estimators`, e `learning_rate` (para XGBoost) ou a profundidade e número de árvores (para Random Forest) precisam ser ajustados cuidadosamente para encontrar um equilíbrio. Um modelo com alta acuracidade no treinamento mas baixa acuracidade no teste é um sinal clássico de overfitting.
    *   **Regressão Linear**: Este modelo é menos propenso ao overfitting devido à sua simplicidade. No entanto, sua capacidade limitada de modelar relações não-lineares pode resultar em menor acuracidade geral, especialmente em dados complexos como séries financeiras. No caso observado, o R-quadrado de 1.00 para a Regressão Linear no conjunto de teste é altamente suspeito e sugere um possível problema (como vazamento de dados ou um erro na configuração da avaliação) que eliminou o trade-off aparente, resultando em uma performance irrealisticamente perfeita no teste. Em um cenário real, esperaríamos um trade-off mais claro onde modelos mais complexos teriam potencial para maior acuracidade, mas também maior risco de overfitting.

In [16]:
from sklearn.metrics import confusion_matrix
import numpy as np

# Criar uma tabela com previsões e valores reais para o XGBoost no conjunto de teste
xgb_comparison_df = test_df[['Data', 'Último']].copy()
xgb_comparison_df['Último_Previsto'] = xgb_predictions

print("XGBoost: Comparação entre Sinal Real e Sinal Previsto no Conjunto de Teste")
display(xgb_comparison_df)

# Para a matriz de confusão, precisamos de classes (sinal).
# Vamos definir um "sinal" simples: se o preço sobe (Último > Último_Lag1).
# Primeiro, precisamos garantir que 'Último_Lag1' está disponível no test_df para calcular o sinal real.
# Se não estiver, precisaremos recalcular ou usar uma abordagem diferente.
# Assumindo que 'Último_Lag1' já foi criado e está no test_df:

# Criar sinal real no test_df
# Precisamos do Último_Lag1 no test_df para calcular a direção do movimento.
# No entanto, as primeiras linhas do test_df não terão um Último_Lag1 válido do train_df.
# Para um cálculo preciso do sinal no test_df, precisaríamos do último valor do Último no train_df
# para calcular o primeiro Último_Lag1 no test_df.
# Uma alternativa simplificada para demonstração é calcular o sinal apenas onde Último_Lag1 está disponível no test_df.

# Calcular sinal real para o test_df
# Para calcular o sinal real corretamente para a primeira entrada no test_df,
# precisamos do último valor do Último do train_df.
last_ultimo_train = train_df['Último'].iloc[-1]

# Criar uma cópia do test_df para evitar SettingWithCopyWarning
test_df_signal = test_df.copy()

# Calculate the real signal for the first row of test_df using the last value of train_df
if not test_df_signal.empty:
    test_df_signal.loc[test_df_signal.index[0], 'Último_Lag1_for_signal'] = last_ultimo_train
    # Fill subsequent NaN values in 'Último_Lag1_for_signal' with the 'Último' of the previous row within test_df
    test_df_signal['Último_Lag1_for_signal'] = test_df_signal['Último'].shift(1).fillna(test_df_signal['Último_Lag1_for_signal'].iloc[0])


# Define real signal based on price change
test_df_signal['Sinal_Real'] = np.where(test_df_signal['Último'] > test_df_signal['Último_Lag1_for_signal'], 1, 0)


# Calcular sinal previsto baseado nas predições do XGBoost
# Precisamos de uma base para a previsão do sinal.
# Uma abordagem seria comparar a previsão com o Último_Lag1 correspondente na linha prevista.
# Isso implica que a previsão para o dia T é comparada com o preço de fechamento do dia T-1.

# Ensure xgb_predictions aligns with test_df_signal for calculating predicted signal
# This assumes xgb_predictions are in the same order as test_df_signal rows.
# To calculate the predicted signal for day T, we compare the predicted price for day T
# with the actual closing price of day T-1 (Último_Lag1_for_signal).
test_df_signal['Sinal_Previsto'] = np.where(xgb_predictions > test_df_signal['Último_Lag1_for_signal'], 1, 0)


# Calcular a Matriz de Confusão
# Precisamos remover as linhas onde o sinal real não pôde ser calculado (e.g., a primeira linha se não tratada)
# Neste caso, como tratamos a primeira linha, podemos usar todo o test_df_signal.
# No entanto, se houver NaN em 'Sinal_Real' ou 'Sinal_Previsto', precisamos removê-los.
test_df_signal.dropna(subset=['Sinal_Real', 'Sinal_Previsto'], inplace=True)


conf_matrix = confusion_matrix(test_df_signal['Sinal_Real'], test_df_signal['Sinal_Previsto'])

print("\nMatriz de Confusão para o Sinal do XGBoost (0=Down/No Change, 1=Up):")
display(conf_matrix)

XGBoost: Comparação entre Sinal Real e Sinal Previsto no Conjunto de Teste


,Data,Último,Último_Previsto
4,2025-08-20,134666.0,133966.062500
5,2025-08-19,134432.0,134954.062500
6,2025-08-18,137322.0,137094.765625
7,2025-08-15,136341.0,135937.468750
8,2025-08-14,136356.0,136768.234375
9,2025-08-13,136687.0,136667.109375
10,2025-08-12,137914.0,137484.671875
11,2025-08-11,135623.0,135752.640625
12,2025-08-08,135913.0,135627.625000
13,2025-08-07,136528.0,136012.359375



Matriz de Confusão para o Sinal do XGBoost (0=Down/No Change, 1=Up):


array([[7, 2],
       [1, 9]])